In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil
import cv2
from tqdm import tqdm          
from random import shuffle
from zipfile import ZipFile
import glob
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import tensorflow as tf

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os

In [8]:
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader
from tflite_model_maker import model_spec
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig

In [11]:
def get_path(directory):
    path = []
    for files in os.listdir(directory):
        path.append(files)
    return path

part_1 = get_path('HAM10000_images_part_1')
part_2 = get_path('HAM10000_images_part_2')

In [12]:
df = pd.read_csv('hmnist_28_28_RGB.csv')
metadata = pd.read_csv('HAM10000_metadata.csv')

In [13]:
def preprocess(folder):
    folder_df = pd.DataFrame(folder, columns = ['filename'])
    folder_df['image_id']=folder_df['filename'].str.slice(0, -4)
    folder_joined = pd.merge(folder_df, metadata, on='image_id', how='inner')[['filename', 'dx']]
    return folder_joined

In [14]:
folder_1 = preprocess(part_1)
folder_2 = preprocess(part_2)

In [15]:
dxs = folder_1['dx'].value_counts().index.to_list()

In [16]:
for i in dxs:
    parent_dir = './'
    path = os.path.join(parent_dir, i)
    os.mkdir(path)

In [18]:
def move_image(df, part):
    files = df['filename']
    dxs  = df['dx']
    for i in tqdm(range(len(files))):
        filename = files[i]
        dx = dxs[i]
        dest_dir = './' + dx
        source_dir = 'HAM10000_images_' + part + '/' + filename
        shutil.move(source_dir, dest_dir)
    return

In [19]:
move_image(folder_1, 'part_1')
move_image(folder_2, 'part_2')

100%|██████████| 5015/5015 [00:02<00:00, 1728.88it/s]


In [20]:
old_names = ['df', 'akiec', 'bkl', 'vasc', 'mel', 'bcc', 'nv']
new_names = ['dermatofibroma', "Bowens_disease", 'benign_keratoses', 'vascular_lesions', 'melanoma', 'basal_cell_carcinoma', 'melanocytic_nevi']

for i in range(len(old_names)):
    os.rename('./'+old_names[i], './'+new_names[i])